In [1]:
import pandas as pd
import json
import numpy as np
import jenks

In [2]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('KOD_MOaMC').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('KOD_OBEC').to_dict(orient='index')

In [3]:
with open('../scratch/okrsky.json', encoding='utf-8') as f:
    jsn = json.load(f)

In [4]:
meta = {}

In [5]:
# zpracovani MC
for mc in momc:
    meta[mc] = {
        'mc': momc[mc]['NAZ_MOaMC'], 
        'obec': momc[mc]['NAZ_OBEC'], 
        'okres': momc[mc]['NAZ_LAU1'], 
        'kraj': momc[mc]['NAZ_CZNUTS3']
    }

In [6]:
# zpracovani obci
for o in ob:
    meta[o] = {
        'mc': None, 
        'obec': ob[o]['NAZ_OBEC'], 
        'okres': ob[o]['NAZ_LAU1'], 
        'kraj': ob[o]['NAZ_CZNUTS3']
    }

In [7]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = momc_kod
    else:
        kd = obec_kod
    
    fe['properties'].update(meta[kd])

In [8]:
okrs = pd.read_csv('../scratch/pet1.csv', sep=';')

In [9]:
okrs['okid'] = okrs.OBEC.astype('str') + '_' + okrs.OKRSEK.astype('str')

In [10]:
cls = [
    'CHYBA',
    'KC_1',
    'KC_2',
    'KC_3',
    'KC_4',
    'HLASY_10',
    'HLASY_11',
    'HLASY_12',
    'HLASY_13',
    'HLASY_14',
    'HLASY_15',
    'HLASY_16',
    'HLASY_17',
    'HLASY_18',
    'HLASY_19',
    'HLASY_20',
    'KC_SUM',
    'KOLO',
    'OBEC',
    'OKRES',
    'OKRSEK',
    'OPRAVA',
    'POSL_KAND',
    'TYP_FORM'
]
okrs.drop(cls, inplace=True, axis=1)

In [11]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [12]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = str(momc_kod) + '_' + str(fe['properties']['Cislo'])
    else:
        kd = str(obec_kod) + '_' + str(fe['properties']['Cislo'])
    
    if kd in okrs:
        fe['properties'].update(okrs[kd])
    else:
        print(kd)

592935_1
545422_1
555177_1
503941_1


In [13]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [14]:
jsn['features'][0]

{'geometry': {'coordinates': [[[[14.384834207332705, 49.96457556907377],
     [14.385453709311202, 49.9643765966084],
     [14.385314241195701, 49.964173566376786],
     [14.385196207492815, 49.96393613829384],
     [14.3850458211357, 49.963586908858964],
     [14.384809760399463, 49.96311205196477],
     [14.384374799525038, 49.96212026206872],
     [14.38474141184349, 49.9621166097775],
     [14.385668400834064, 49.962190095116505],
     [14.38688199875932, 49.96237078511879],
     [14.38802376814408, 49.96216384916892],
     [14.389188831942603, 49.95441633828891],
     [14.388039857042868, 49.949857407461685],
     [14.388037191555432, 49.9498567162355],
     [14.38803495379446, 49.94986060141055],
     [14.387911579317764, 49.949832367705085],
     [14.387846090603865, 49.9499017801614],
     [14.387716139596412, 49.94989311060708],
     [14.387579248648859, 49.94988400431829],
     [14.387392110836435, 49.94987831913232],
     [14.387229277402975, 49.94987343693474],
     [14.386

In [16]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w')
target.truncate()
target.write('var breaks = {\n')

for i in range(1, 10):
    print('kandidat ' + str(i))
    vals = []
    for ftr in jsn['features']:
        try:
            vals.append(ftr['properties']['HLASY_' + str(i).rjust(2, '0')] / ftr['properties']['PL_HL_CELK'])
        except:
            print('omit')
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('"HLASY_' + str(i).rjust(2, '0') + '": ' + str(brks) + ',\n')
        
vals = []
print('ucast')
for ftr in jsn['features']:
    try:
        vals.append(ftr['properties']['PL_HL_CELK'] / ftr['properties']['VOL_SEZNAM'])
    except:
        print('omit')

vals = np.trim_zeros(vals)
if np.mean(vals) == 0:
    pass
else:
    brks = [0, 0, 0, 0]
    try:
        casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
        brks = casp.breaks
    except:
        pass
    target.write('"UCAST": ' + str(brks) + ',\n')       
target.write("};")
target.close()

kandidat 1
omit
omit
omit
omit
Running Jenks-Caspall natural breaks...
**Attempt number 1
Step 1 : kmeans++ initalization, GVF = 0.914610
Step 2 : kmeans complete in 55 iterations and 8745 moves, GVF = 0.941390
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941390
Running Jenks-Caspall natural breaks...
**Attempt number 2
Step 1 : kmeans++ initalization, GVF = 0.935378
Step 2 : kmeans complete in 51 iterations and 3751 moves, GVF = 0.941384
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941384
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.939466
Step 2 : kmeans complete in 12 iterations and 1039 moves, GVF = 0.941390
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941390
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.924951
Step 2 : kmeans complete in 49 iterations and 4728 moves, GVF = 0.941384
Step 3 : Forcing completed in 1 cycles and 0 mov

Step 2 : kmeans complete in 23 iterations and 2585 moves, GVF = 0.941277
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941277
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.937514
Step 2 : kmeans complete in 41 iterations and 2930 moves, GVF = 0.941279
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941279
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.927059
Step 2 : kmeans complete in 28 iterations and 3615 moves, GVF = 0.941279
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.941279
Jenks-Caspall competed!
14751 values, 5 classes
Breaks with bounds [0.0028735632183908046, 0.06722689075630252, 0.12894736842105264, 0.18705035971223022, 0.24922118380062305, 0.47435897435897434]
Sum of within classes variances 5
Between classes variance 88
Goodness of variance fit 0.941279
kandidat 8
omit
omit
omit
omit
Running Jenks-Caspall natural breaks...
**A